In [27]:
# http://www.slideviper.oquanta.info/tutorial/slideshow_tutorial_slides.html#/3
# http://www.damian.oquanta.info/posts/make-your-slides-with-ipython.html
# https://github.com/damianavila/slideviper_test/blob/gh-pages/tutorial/slideshow_tutorial.ipynb
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
              'theme': 'serif',
              'transition': 'zoom',
              'start_slideshow_at': 'selected',
})

{u'start_slideshow_at': 'selected', u'theme': 'serif', u'transition': 'zoom'}

# Monasca Bootcamp

[Roland Hochmuth](https://www.linkedin.com/in/rolandhochmuth)

[Michael Hoppal](https://www.linkedin.com/in/hoppalmichael)

# Agenda

* Overview
* Architecture
* Deployment
* API, CLI (python-monascaclient) and client
* Agent
* Developing
* Current status
* What next?


# Who is involved in the project?

* Hewlett Packard Enterprise

* Time Warner Cable

* Fujitsu

* Cisco

* NEC

* Cray

* SAP

# Import libraries

In [28]:
import datetime
import time

# Import libraries use for visualization and analysis
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import numpy as np

# Import the Monasca and Keystone client
from monascaclient import client
from monascaclient import ksclient

# Initialize the Keystone and Monasca Client

In [29]:
KEYSTONE_URL = 'http://192.168.10.6:5000/v3'
PROJECT_NAME = 'mini-mon'
USERNAME = 'mini-mon'
PASSWORD = 'password'

In [30]:
# Authenticate to Keystone
keystone_client = ksclient.KSClient(auth_url=KEYSTONE_URL, username=USERNAME, password=PASSWORD)

# Create the Monasca client
monasca_client = client.Client('2_0', keystone_client.monasca_url, token=keystone_client.token)

# Initialize environment variables to use the Monasca CLI
%env OS_PROJECT_NAME=$PROJECT_NAME
%env OS_PASSWORD=$PASSWORD
%env OS_AUTH_URL=$KEYSTONE_URL
%env OS_USERNAME=$USERNAME

# If you are doing development on your local system and running from there then set the URL to the Monasca API
# to override the one returned from Keystone
# %env MONASCA_API_URL=http://127.0.0.1:8070/v2.0

env: OS_PROJECT_NAME=mini-mon
env: OS_PASSWORD=password
env: OS_AUTH_URL=http://192.168.10.6:5000/v3
env: OS_USERNAME=mini-mon


# Initialize Plotly

We'll be using Plotly for displaying some graphs later on in this notebook.

In [31]:
WIDTH = 640
HEIGHT = 768

layout = go.Layout(
    autosize=False,
    width=WIDTH,
    height=HEIGHT,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ))

# List Metrics

In [32]:
!monasca help metric-list

usage: monasca metric-list [--name <METRIC_NAME>]
                           [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                           [--starttime <UTC_START_TIME>]
                           [--endtime <UTC_END_TIME>]
                           [--offset <OFFSET LOCATION>]
                           [--limit <RETURN LIMIT>]

List metrics for this tenant.

Optional arguments:
  --name <METRIC_NAME>  Name of the metric to list.
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (pre

# List all metrics

In [33]:
!monasca metric-list

+-----------------------------+---------------------------------+
| name                        | dimensions                      |
+-----------------------------+---------------------------------+
| cpu.idle_perc               | hostname: devstack              |
|                             | service: monitoring             |
| cpu.stolen_perc             | hostname: devstack              |
|                             | service: monitoring             |
| cpu.system_perc             | hostname: devstack              |
|                             | service: monitoring             |
| cpu.user_perc               | hostname: devstack              |
|                             | service: monitoring             |
| cpu.wait_perc               | hostname: devstack              |
|                             | service: monitoring             |
| disk.inode_used_perc        | device: vagrant-cache           |
|                             | hostname: devstack              |
|         

# List metrics and filter on name, dimensions and starttime

In [34]:
!monasca metric-list --name cpu.user_perc --dimensions hostname=devstack --starttime -60


+---------------+---------------------+
| name          | dimensions          |
+---------------+---------------------+
| cpu.user_perc | hostname: devstack  |
|               | service: monitoring |
+---------------+---------------------+


# Create a function to get metrics using the Monasca client

In [35]:
def get_metrics(names = [None], dimensions = None, limit=10):
    metrics = []
    for name in names:
        kwargs = {}
        if name is not None:
            kwargs['name'] = name
        if dimensions is not None:
            kwargs['dimensions'] = dimensions
        kwargs['limit'] = limit
        metrics = metrics + monasca_client.metrics.list(**kwargs)
    return metrics

# Query measurements

In [36]:
!monasca help measurement-list

usage: monasca measurement-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                [--endtime <UTC_END_TIME>]
                                [--offset <OFFSET LOCATION>]
                                [--limit <RETURN LIMIT>] [--merge_metrics]
                                <METRIC_NAME> <UTC_START_TIME>

List measurements for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to list measurements.
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
             

# Query measurement list

In [37]:
!monasca measurement-list --dimensions hostname=devstack cpu.user_perc -120

+---------------+---------------------+--------------------------+--------------+------------+
| name          | dimensions          | timestamp                | value        | value_meta |
+---------------+---------------------+--------------------------+--------------+------------+
| cpu.user_perc | hostname: devstack  | 2016-04-08T23:01:41.000Z |        3.900 |            |
|               | service: monitoring | 2016-04-08T23:01:56.000Z |        4.400 |            |
|               |                     | 2016-04-08T23:02:11.000Z |        4.100 |            |
|               |                     | 2016-04-08T23:02:26.000Z |        4.100 |            |
|               |                     | 2016-04-08T23:02:41.000Z |        4.300 |            |
|               |                     | 2016-04-08T23:02:56.000Z |        3.800 |            |
|               |                     | 2016-04-08T23:03:11.000Z |        4.500 |            |
|               |                     | 2016-04-08

# Create a function to get measurements using the Monasca Client

In [38]:
def get_measurements(metrics, start_time = None, end_time = None, limit=None):
    measurements = []
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=3600)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time
        
        # Invoke the monasca client
        measurements.append(monasca_client.metrics.list_measurements(**kwargs))
        
    return measurements

In [39]:
metrics = get_metrics(['cpu.user_perc'])
get_measurements(metrics)

[[{u'columns': [u'timestamp', u'value', u'value_meta'],
   u'dimensions': {u'hostname': u'devstack', u'service': u'monitoring'},
   u'id': u'2016-04-09T00:57:45.000Z',
   u'measurements': [[u'2016-04-09T00:01:42.000Z', 4.2, {}],
    [u'2016-04-09T00:01:57.000Z', 5, {}],
    [u'2016-04-09T00:02:12.000Z', 4.2, {}],
    [u'2016-04-09T00:02:27.000Z', 4.3, {}],
    [u'2016-04-09T00:02:42.000Z', 4.1, {}],
    [u'2016-04-09T00:02:57.000Z', 4.1, {}],
    [u'2016-04-09T00:03:12.000Z', 4.3, {}],
    [u'2016-04-09T00:03:27.000Z', 3.7, {}],
    [u'2016-04-09T00:03:42.000Z', 4, {}],
    [u'2016-04-09T00:03:57.000Z', 4.1, {}],
    [u'2016-04-09T00:04:12.000Z', 4.4, {}],
    [u'2016-04-09T00:04:27.000Z', 4.7, {}],
    [u'2016-04-09T00:04:42.000Z', 4.6, {}],
    [u'2016-04-09T00:04:57.000Z', 4.6, {}],
    [u'2016-04-09T00:05:12.000Z', 4, {}],
    [u'2016-04-09T00:05:27.000Z', 4.5, {}],
    [u'2016-04-09T00:05:42.000Z', 3.7, {}],
    [u'2016-04-09T00:05:57.000Z', 4.2, {}],
    [u'2016-04-09T00:06:12.00

# Create a function to translate measurements to a Pandas DataFrame¶

In [40]:
def df_from_measurements(measurements):
    '''Returns a DataFrame given measurements'''
    measurement = measurements[0][0]
    m = np.array(measurement['measurements'])  
    timestamps = m[:, measurement['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)  
    
    for measurement in measurements:
        measure = measurement[0]
        m = np.array(measure['measurements'])
        name = measure['name']
        df[name] = m[:, measure['columns'].index('value')]     
    return df

# Query measurements using the Monasca client


In [41]:
#metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
measurements = get_measurements(metrics)

# Display measurements using Plotly

In [42]:
df = df_from_measurements(measurements)
df.iplot(subplots=True, shape=(len(df.columns),1), shared_xaxes=True, fill=True)
#df.iplot(kind='line', fill=True,
#         xTitle='Date',
#         yTitle='Value',
#         title='Statistics',
#         layout=layout)
#df.scatter_matrix(world_readable=True)

# Get statistics

In [43]:
!monasca help metric-statistics

usage: monasca metric-statistics [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                 [--endtime <UTC_END_TIME>]
                                 [--period <PERIOD>]
                                 [--offset <OFFSET LOCATION>]
                                 [--limit <RETURN LIMIT>] [--merge_metrics]
                                 <METRIC_NAME> <STATISTICS> <UTC_START_TIME>

List measurement statistics for the specified metric.

Positional arguments:
  <METRIC_NAME>         Name of the metric to report measurement statistics.
  <STATISTICS>          Statistics is one or more (separated by commas) of
                        [AVG, MIN, MAX, COUNT, SUM].
  <UTC_START_TIME>      measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR Format: -120 (previous 120
                        minutes).

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
    

# Create a function to get statistics using the Monasca client

In [44]:
def get_statistics(metrics, statistics=['avg'], interval = 3600, start_time = None, end_time = None):
    statistics_list = []
    
    period = interval / 512
    
    if period < 60:
        period = 60
        
    period = period - period%60
    
    if start_time == None:
        start_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=interval)
        start_time = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")

    if end_time == None:
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=0)
        end_time = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
    for metric in metrics:
        kwargs = {}
        kwargs['statistics'] = statistics
        kwargs['period'] = period
        kwargs['name'] = metric['name']
        kwargs['dimensions'] = metric['dimensions']
        kwargs['start_time'] = start_time
        kwargs['end_time'] = end_time            
        statistics_list.append(monasca_client.metrics.list_statistics(**kwargs))
        
    return statistics_list

# Create a function to translate statistics to a Pandas DataFrame

In [45]:
def df_from_statistics(statistics, fn = 'avg'):
    '''Returns a DataFrame given statistics'''
    stat = statistics[0][0]
    m = np.array(stat['statistics'])  
    timestamps = m[:, stat['columns'].index('timestamp')]
    df = pd.DataFrame(index = timestamps)
    
    for statistic in statistics:
        stat = statistic[0]
        m = np.array(stat['statistics'])
        name = stat['name']
        df[name] = m[:, stat['columns'].index('avg')]
    return df

# Query statistics using the Monasca client

In [46]:
#metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
statistics = get_statistics(metrics, ['avg'], 3600)


# Display statistics using Plotly

In [47]:
df = df_from_statistics(statistics)
df.iplot(subplots=True, shape=(len(df.columns),1), shared_xaxes=True, fill=True)
#df.iplot(kind='line', fill=True,
#         xTitle='Date',
#         yTitle='Value',
#         title='Statistics',
#         layout=layout)
#df.scatter_matrix(world_readable=True)

In [48]:
metrics = get_metrics(names=['cpu.user_perc', 'cpu.system_perc'])
metrics = get_metrics()
statistics = get_statistics(metrics, ['avg'], 3600)
df = df_from_statistics(statistics)
#df.iplot(subplots=True, shape=(len(df.columns),1), shared_xaxes=True, fill=True,
df.iplot(kind='line', fill=True,
         xTitle='Date',
         yTitle='Value',
         title='Statistics',
         layout=layout)
df.scatter_matrix(world_readable=True)

In [49]:
df.iplot(kind='box', layout=layout)

# Query the Alarm History

In [50]:
!monasca help alarm-history-list

usage: monasca alarm-history-list [--dimensions <KEY1=VALUE1,KEY2=VALUE2...>]
                                  [--starttime <UTC_START_TIME>]
                                  [--endtime <UTC_END_TIME>]
                                  [--offset <OFFSET LOCATION>]
                                  [--limit <RETURN LIMIT>]

List alarms state history.

Optional arguments:
  --dimensions <KEY1=VALUE1,KEY2=VALUE2...>
                        key value pair used to specify a metric dimension.
                        This can be specified multiple times, or once with
                        parameters separated by a comma. Dimensions need
                        quoting when they contain special chars
                        [&,(,),{,},>,<] that confuse the CLI parser.
  --starttime <UTC_START_TIME>
                        measurements >= UTC time. format:
                        2014-01-01T00:00:00Z. OR format: -120 (previous 120
                        minutes).
  --endtime <UTC_END_TIME>

# Query the alarm history using the Monasca client

In [51]:
monasca_client.alarms.history_list(**{})

[]

# Developing and Testing

* Python and Java codebase

* Monasca DevStack Plugin

* Unit Tests

* Monasca Tempest Tests

* Monasca and OpenStack CI

# Monasca Repos

* Monasca is a micro-services message bus based architecture.

* Several repos:

    * monasca-api: both Python and Java
    
    * monasca-persister: both Python and Java
    
    * monasca-thresh: Java
    
    * monasca-notification: Python
    
    * monasca-common: both Python and Java
    
    * monasca-agent: Python
    
    * monasca-statsd: Python
    
    * monasca-ui: Python
    
    * python-monascaclient: Python

# Monasca DevStack Plugin

* DevStack is the primary developmement environment for OpenStack.

    * See http://docs.openstack.org/developer/devstack/

* The Monasca DevStack plugin installs the Monasca Service, Agent, Horizon Panel, and Grafana

* README at, https://github.com/openstack/monasca-api/tree/master/devstack

* Best way to get started is to install Vagrant and the Vagrantfile at, https://github.com/openstack/monasca-api/blob/master/devstack/Vagrantfile.

# Monasca Tempest Tests

* [Tempest](http://docs.openstack.org/developer/tempest) is the integration test suite for OpenStack.

* Tempest has an external [Test Plugin Interface](http://docs.openstack.org/developer/tempest/plugin.html) interface that enables anyone to integrate an external test suite.

* There is a [Monasca Tempest Plugin](https://github.com/openstack/monasca-api/tree/master/monasca_tempest_tests)

* Currently, there are around 150 Tempest Tests written for Monasca that run in around 5 minutes.

# Monasca CI

* Monasca is fully integrated in the OpenStack CI system.

* Gated jobs are run with each commit on both the Python and Java components through the normal gates and all the Monasca Tempest tests using the Monasca DevStack Plugin.

* Currently, Java is non-voting